In [1]:
# Import all necessary libraries.

import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
# Problem Practice 1 

df_book = pd.read_csv("CharlesBookClub.csv")
df_book = df_book.drop(["Seq#", "ID#", "Gender", "M", "R", "F", "Mcode", "Rcode", "Fcode",
                        "Yes_Florence", "No_Florence", "Florence",
                          "FirstPurch", "Related Purchase"], axis=1)
df_book = df_book.astype(bool)
df_book.head(10)

,ChildBks,YouthBks,CookBks,DoItYBks,RefBks,ArtBks,GeogBks,ItalCook,ItalAtlas,ItalArt
0,False,True,True,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,True,True,True,False,True,False,True,True,False,False
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,True,False,False,False
7,True,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False
9,False,False,True,False,False,False,False,False,False,False


In [8]:
book_list = ["ChildBks", "YouthBks", "CookBks", "DoItYBks", "RefBks", "ArtBks",
             "GeogBks", "ItalCook", "ItalAtlas", "ItalArt"]
for feature in book_list:
    df_book[feature] = df_book[feature].map({True: 1, False: 0})
df_book.head(10)

,ChildBks,YouthBks,CookBks,DoItYBks,RefBks,ArtBks,GeogBks,ItalCook,ItalAtlas,ItalArt
0,0,1,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,1,1,1,0,1,0,1,1,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,1,0,0,0
7,1,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0
9,0,0,1,0,0,0,0,0,0,0


In [9]:
frequent_itemsets = apriori(df_book, min_support=0.1, use_colnames=True)
rules = association_rules(frequent_itemsets, num_itemsets=10, metric="confidence", min_threshold=0.5)

c:\Users\coryg\AppData\Local\Programs\Python\Python310\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [10]:
rules = rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]
rules = rules[(rules['support'] >= 0.1) &
              (rules['confidence'] >= 0.5) &
              (rules['lift'] >= 2)]

rules = rules[rules['consequents'].apply(lambda x: len(x) == 1)]
rules = rules.sort_values('lift', ascending=[False])

In [11]:
rules

,antecedents,consequents,support,confidence,lift
15,"(ChildBks, CookBks)",(DoItYBks),0.12775,0.527893,2.072198


Result: Recommend Do It Yourself books if Child Books and Cookbooks are bought.

In [12]:
# Problem Practice 2 

df_movie = pd.read_csv("movierating.csv")
df_movie = df_movie.drop("User", axis=1)
df_movie.head(10)

,movie1,movie2,movie3,movie4,movie5,movie6,movie7,movie8,movie9,movie10,...,movie91,movie92,movie93,movie94,movie95,movie96,movie97,movie98,movie99,movie100
0,3.0,3.0,3.0,3.0,0.0,1.0,0.0,3.0,1.0,2.0,...,4.0,1.0,0.0,4.0,4.0,3.0,2.0,2.0,2.0,2.0
1,4.0,4.0,0.0,0.0,4.0,4.0,1.0,4.0,2.0,2.0,...,2.0,0.0,3.0,2.0,4.0,3.0,4.0,2.0,4.0,0.0
2,4.0,1.0,0.0,4.0,2.0,4.0,2.0,0.0,3.0,2.0,...,1.0,3.0,4.0,2.0,2.0,1.0,0.0,3.0,1.0,2.0
3,1.0,2.0,3.0,0.0,3.0,0.0,1.0,3.0,4.0,1.0,...,3.0,4.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0
4,2.0,4.0,2.0,3.0,1.0,1.0,2.0,2.0,4.0,4.0,...,1.0,4.0,2.0,4.0,2.0,4.0,1.0,1.0,4.0,2.0
5,4.0,1.0,2.0,2.0,4.0,3.0,0.0,1.0,1.0,2.0,...,1.0,3.0,3.0,1.0,1.0,3.0,0.0,0.0,0.0,2.0
6,3.0,4.0,2.0,0.0,1.0,3.0,3.0,4.0,1.0,4.0,...,3.0,3.0,1.0,3.0,1.0,1.0,1.0,2.0,0.0,2.0
7,4.0,0.0,3.0,1.0,2.0,3.0,4.0,4.0,2.0,0.0,...,3.0,2.0,3.0,3.0,1.0,4.0,1.0,1.0,1.0,3.0
8,3.0,1.0,4.0,2.0,2.0,0.0,0.0,2.0,4.0,4.0,...,2.0,1.0,4.0,0.0,3.0,4.0,1.0,1.0,4.0,0.0
9,1.0,1.0,2.0,2.0,1.0,1.0,0.0,3.0,2.0,0.0,...,3.0,3.0,4.0,4.0,0.0,4.0,4.0,0.0,1.0,0.0


In [14]:
movie_similarity = cosine_similarity(df_movie.values)
print(movie_similarity)
movie_sim = pd.DataFrame(movie_similarity, index=df_movie.index, columns=df_movie.index)
movie_sim

[[1.         0.69114854 0.69942487 ... 0.61859463 0.69466797 0.68573345]
 [0.69114854 1.         0.68217941 ... 0.64840539 0.6563425  0.66635677]
 [0.69942487 0.68217941 1.         ... 0.66944752 0.72965145 0.67602629]
 ...
 [0.61859463 0.64840539 0.66944752 ... 1.         0.71363564 0.621508  ]
 [0.69466797 0.6563425  0.72965145 ... 0.71363564 1.         0.63101195]
 [0.68573345 0.66635677 0.67602629 ... 0.621508   0.63101195 1.        ]]


,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,1.000000,0.691149,0.699425,0.632598,0.676989,0.673564,0.707842,0.713161,0.784518,0.627706,...,0.674759,0.684459,0.678719,0.590243,0.700486,0.638030,0.637853,0.618595,0.694668,0.685733
1,0.691149,1.000000,0.682179,0.556659,0.625040,0.636291,0.644435,0.664582,0.645689,0.626489,...,0.640688,0.667208,0.673816,0.596877,0.772291,0.585976,0.619141,0.648405,0.656342,0.666357
2,0.699425,0.682179,1.000000,0.615776,0.739385,0.742187,0.737161,0.731368,0.716411,0.688660,...,0.793910,0.703710,0.731180,0.652186,0.756475,0.710356,0.624361,0.669448,0.729651,0.676026
3,0.632598,0.556659,0.615776,1.000000,0.636691,0.593230,0.657467,0.675136,0.626613,0.686145,...,0.627105,0.656940,0.603218,0.562922,0.644477,0.665081,0.598960,0.656468,0.590720,0.617916
4,0.676989,0.625040,0.739385,0.636691,1.000000,0.650961,0.760325,0.748662,0.722769,0.717583,...,0.720344,0.665266,0.684689,0.660010,0.698906,0.698762,0.652187,0.613950,0.725453,0.732917
5,0.673564,0.636291,0.742187,0.593230,0.650961,1.000000,0.652508,0.638159,0.665987,0.664769,...,0.652220,0.722968,0.742760,0.603311,0.689556,0.645319,0.669389,0.673600,0.656494,0.678275
6,0.707842,0.644435,0.737161,0.657467,0.760325,0.652508,1.000000,0.734714,0.695456,0.620752,...,0.672301,0.671960,0.667802,0.627075,0.714065,0.721554,0.586860,0.651264,0.663284,0.740074
7,0.713161,0.664582,0.731368,0.675136,0.748662,0.638159,0.734714,1.000000,0.692005,0.674210,...,0.739481,0.661393,0.646361,0.603657,0.716327,0.742647,0.617466,0.638709,0.653427,0.742094
8,0.784518,0.645689,0.716411,0.626613,0.722769,0.665987,0.695456,0.692005,1.000000,0.670693,...,0.720201,0.696273,0.684075,0.591102,0.684671,0.630598,0.700553,0.646879,0.654981,0.651645
9,0.627706,0.626489,0.688660,0.686145,0.717583,0.664769,0.620752,0.674210,0.670693,1.000000,...,0.685586,0.723024,0.673609,0.532114,0.672934,0.668070,0.645343,0.649012,0.674279,0.723754


In [15]:
cosine_threshold = 0.75
sim_pairs = []
for user_a in movie_sim.index:
    for user_b in movie_sim.columns:
        if (user_a != user_b) and (movie_sim.loc[user_a, user_b] > cosine_threshold):
            sim_pairs.append((user_a+1, user_b+1))

sim_pairs

[(1, 9),
 (1, 11),
 (2, 45),
 (3, 21),
 (3, 30),
 (3, 39),
 (3, 41),
 (3, 45),
 (5, 7),
 (5, 30),
 (6, 40),
 (7, 5),
 (8, 25),
 (8, 30),
 (9, 1),
 (9, 30),
 (11, 1),
 (12, 50),
 (21, 3),
 (21, 39),
 (21, 40),
 (22, 45),
 (25, 8),
 (29, 50),
 (30, 3),
 (30, 5),
 (30, 8),
 (30, 9),
 (36, 39),
 (39, 3),
 (39, 21),
 (39, 36),
 (40, 6),
 (40, 21),
 (41, 3),
 (45, 2),
 (45, 3),
 (45, 22),
 (50, 12),
 (50, 29)]

In [18]:
for iter in range(0, len(sim_pairs)):
    if sim_pairs[iter][0] == 5:
        print(sim_pairs[iter])

(5, 7)
(5, 30)


Result: We can observe that user 5 is related to user 7 and user 30 from the above
code results.

In [23]:
prediction = movie_sim.loc[4, 6] * df_movie.loc[6][35] + movie_sim.loc[4, 29] * df_movie.loc[29][35]
sum_pred = movie_sim.loc[4, 6] + movie_sim.loc[4, 29]

weighted = prediction/sum_pred
print(weighted)

2.0


C:\Users\coryg\AppData\Local\Temp\ipykernel_3404\1682282470.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  prediction = movie_sim.loc[4, 6] * df_movie.loc[6][35] + movie_sim.loc[4, 29] * df_movie.loc[29][35]


Hence the predicted rating for movie 36 by user 5 is 2.0 as shown above.

In [ ]:
# Survey for Extra Credit 

# 1. How long did it take to finish this homework?

# It took about a week and a couple days to complete this homework since I did 
# my calculations for the first theory problem by hand. These were not difficult 
# but were just tedious to complete. I now rethink and maybe should have done 
# this part using basic Python calculations. 

# 2. What problem was the easiest to implement?

# The easiest problem to implement was the cosine similarity problem since
# we only had to use the built-in library to return the similarity matrix. 

# 3. What problem was the most difficult to implement? 

# The most difficult problem to implement was the theory problem 1 because
# I kept making mistakes on the calculations for the cosine similarity.movie_similarity.
    
# 4. Do you have the confidence to implement all these algorithms to your real 
# problems in the future? 

# I sort of have the confidence but I think with enough practice, I think I can
# implement these recommendation algorithms to real life problems as I find them
# quite interesting.